In [13]:
import os
from datasets import load_dataset
from transformers import (
    AutoTokenizer
)

from omegaconf import OmegaConf
from omegaconf.dictconfig import DictConfig

import datasets
import transformers
from accelerate import Accelerator


In [3]:
base_url = "/mnt/share/the_pile/"
data_files = {
    "train": [base_url + "train/" + f"{idx:02d}.jsonl.zst" for idx in range(1)],
}

In [4]:
data_files

{'train': ['/mnt/share/the_pile/train/00.jsonl.zst']}

In [12]:
for subset in data_files.keys():
    for _file in data_files[subset]:
       write_tokenized_file(_file)

tokenized00.josnl


In [11]:
def write_tokenized_file(_file):
    tokenized_path = '/mnt/share/tokenized_pile'
    # tokenized_filename = _file...
    tokenized_filename = "tokenized_"+_file[_file.find('.')-2:_file.find('.')]+ ".josnl"
    print(tokenized_filename)
    return
    

In [33]:
cfg = OmegaConf.load('conf/config.yaml')

In [30]:
def load_tokenizer(cfg: DictConfig):
    if cfg.tokenizer.name is not None:
        tokenizer = AutoTokenizer.from_pretrained(
            cfg.tokenizer.name, use_fast=cfg.tokenizer.use_fast
        )
    else:
        tokenizer = AutoTokenizer.from_pretrained(
            cfg.model.name, use_fast=cfg.tokenizer.use_fast
        )
    
    if tokenizer.pad_token is None and tokenizer.eos_token is not None:
        tokenizer.pad_token = tokenizer.eos_token

    return tokenizer

In [34]:
tokenizer = load_tokenizer(cfg)

Using pad_token, but it is not set yet.


In [35]:
tokenizer

GPT2TokenizerFast(name_or_path='EleutherAI/gpt-j-6b', vocab_size=50257, model_max_length=2048, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True)

In [12]:
from transformers import LlamaTokenizer

tokenizer = LlamaTokenizer.from_pretrained("decapoda-research/llama-7b-hf", config="/mnt/share/llama-7b-hf/tokenizer_config.json")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


In [19]:
from transformers import LlamaTokenizer

def load_tokenizer():

    tokenizer = LlamaTokenizer.from_pretrained("decapoda-research/llama-7b-hf", config="/mnt/share/llama-7b-hf/tokenizer_config.json")
    # cache_dir='/mnt/share-4tb/.cache/huggingface/hub'
    # tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    return tokenizer

In [ ]:
def load_tokenizer(cfg: DictConfig):
    if cfg.tokenizer.name is not None:
        tokenizer = AutoTokenizer.from_pretrained(
            cfg.tokenizer.name, use_fast=cfg.tokenizer.use_fast
        )
    else:
        tokenizer = AutoTokenizer.from_pretrained(
            # cfg.model.name, use_fast=cfg.tokenizer.use_fast
            cfg.model.name
        )
    
    if tokenizer.pad_token is None and tokenizer.eos_token is not None:
        tokenizer.pad_token = tokenizer.eos_token

    return tokenizer

In [23]:
tokenizer.pad_token = tokenizer.eos_token

In [20]:
tokenizer = load_tokenizer()  

TypeError: 'NoneType' object is not callable

In [ ]:
def load_tokenizer(cfg: DictConfig):
    if cfg.tokenizer.name is not None:
        tokenizer = AutoTokenizer.from_pretrained(
            cfg.tokenizer.name, use_fast=cfg.tokenizer.use_fast
        )
    else:
        tokenizer = AutoTokenizer.from_pretrained(
            # cfg.model.name, use_fast=cfg.tokenizer.use_fast
            cfg.model.name
        )
    
    if tokenizer.pad_token is None and tokenizer.eos_token is not None:
        tokenizer.pad_token = tokenizer.eos_token

    return tokenizer

def load_datasets(cfg: DictConfig, logger):

    # base_url = "https://the-eye.eu/public/AI/pile/"
    base_url = '/mnt/share-4tb/the_pile/'
    data_files = {
        "train": [base_url + "train/" + f"{idx:02d}.jsonl.zst" for idx in range(1)],
        "eval": base_url + "val.jsonl.zst",
        # "test": base_url + "test.jsonl.zst",
    }

    # data_files
    # from datasets import load_dataset

    pile_dataset_streamed = load_dataset("json", data_files=data_files, streaming=True)

    def cast_to_tensor(_dict):
        _tmp = {k: torch.tensor(v, dtype=torch.long) for k, v in _dict.items() if k in ['input_ids', 'attention_mask', 'labels']}
        return _tmp

    # first option
    def add_labels(example):
        return {"labels": example["input_ids"]}
    
    # def combine(example):
    #     _dict = {k: torch.tensor(v, dtype=torch.long) for k, v in example.items() if k in ['input_ids', 'attention_mask']}
    #     _dict["labels"] = _dict["input_ids"]
    #     return _dict

    tokenizer = load_tokenizer(cfg)    
    # tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
    # tokenizer = AutoTokenizer.from_pretrained('gpt2')
    # if tokenizer.pad_token is None and tokenizer.eos_token is not None:
    #     tokenizer.pad_token = tokenizer.eos_token

    train_tokenized_dataset = pile_dataset_streamed['train'].map(lambda x: tokenizer(x["text"], padding="max_length", truncation=True, max_length=256), batched=True, batch_size=3_200)
    train_tokenized_dataset = train_tokenized_dataset.remove_columns(['text', 'meta'])
    # train_tokenized_dataset = train_tokenized_dataset.map(lambda x: combine(x))
    train_tokenized_dataset = train_tokenized_dataset.map(lambda x: add_labels(x))
    # train_tokenized_dataset = train_tokenized_dataset.map(lambda x: cast_to_tensor(x))

    eval_tokenized_dataset = pile_dataset_streamed['eval'].map(lambda x: tokenizer(x["text"], padding="max_length", truncation=True, max_length=256), batched=True, batch_size=3_200)
    eval_tokenized_dataset = eval_tokenized_dataset.remove_columns(['text', 'meta'])
    eval_tokenized_dataset = eval_tokenized_dataset.map(lambda x: add_labels(x))
    eval_tokenized_dataset = eval_tokenized_dataset.map(lambda x: cast_to_tensor(x))


    train_tokenized_dataset = train_tokenized_dataset.shuffle(seed=cfg.training.seed, buffer_size=6_400)
    eval_tokenized_dataset = eval_tokenized_dataset.shuffle(seed=cfg.training.seed, buffer_size=6_400)
    # print(next(iter(tokenized_dataset_tensor)).keys())
    # status = True
    # i = 0
    # while status:
    #     file_name = cfg.dataset.file_name + "_" + str(i)
    #     data_file = os.path.join(cfg.dataset.data_dir, file_name)

    #     try:
    #         tokenized_dataset_batch = load_from_disk(data_file)
    #         tokenized_dataset_batch.set_format(type='pt')
    #         logger.info(f"loaded data from {data_file}.")
    #     except:
    #         status = False
    #         logger.info(f"{data_file} doesn't exist.")

    #     if i==0:
    #         tokenized_dataset = tokenized_dataset_batch
    #     else:
    #         tokenized_dataset = concatenate_datasets([tokenized_dataset, tokenized_dataset_batch])
        
    #     i += 1

    return train_tokenized_dataset, eval_tokenized_dataset

In [10]:
train_dataset[5]

{'text': "Q:\n\nWhy was Mundungus banned from the Hog's Head?\n\nIn Order of the Phoenix while the trio were in the Hogs Head for the first time plotting the start of Dumbledore's Army, it transpires that ol' Dung was lurking in the pub in a disguise, having been banned 20 years previously according to Sirius. \nFirstly, why was he banned? this could possibly be the tight spot that Albus had helped Dung with in the first place that made him loyal to Albus.  \nAnd secondly, how is it that he is then speaking to Aberforth in Halfblood Prince? (assuming the ban was for something rather unforgivable, 20 years is a long time?) \nThey both could have been in the Order by then, but unlikely given Aberforth's attitude in Deathly Hallows once the trio arrive in Hogsmeade looking for the tiara.  We learn now that a lot of trafficking goes on through the Hogs Head so maybe Dung was trading with Aberforth, Sirius' mirror and various other Black artifacts, he just was not allowed in the pub. \nAnyo

In [9]:
train_dataset = datafile['train']

In [5]:
datafile = load_dataset("json", data_files=data_files, cache_dir='/mnt/share/.cache/huggingface/datasets')
# This ran successfully with train/00.jsonl.zst in 27 min I think

In [6]:
datafile = load_dataset("json", data_files=data_files, cache_dir='/mnt/share/.cache/huggingface/datasets')
# Running this with dataset cache
# 9:43 to complete.
# Multiplied by 30 files: 4 hrs: 50 min just to load the data from cache

Found cached dataset json (/mnt/share/.cache/huggingface/datasets/json/default-73de0422a3395cc6/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e)
100%|██████████| 1/1 [09:43<00:00, 583.41s/it]


In [17]:
datafile_copy = datafile

In [11]:
train_dataset = datafile['train']

In [ ]:
tokenizer

In [15]:
def create_accelerator(cfg: DictConfig) -> Accelerator:
    accelerator = (
        Accelerator(log_with=cfg.tracking.report_to, project_dir=cfg.output_dir)
        if cfg.tracking.enabled
        else Accelerator()
    )
    if accelerator.is_local_main_process:
        datasets.utils.logging.set_verbosity_warning()
        transformers.utils.logging.set_verbosity_info()
    else:
        datasets.utils.logging.set_verbosity_error()
        transformers.utils.logging.set_verbosity_error()

    return accelerator

In [16]:
def preprocess(cfg, accelerator, tokenizer, raw_datasets):

    # First we tokenize all the texts.
    column_names = raw_datasets.column_names
    text_column_name = "text" if "text" in column_names else column_names["train"][0]
    if cfg.dataset.concatenate_raw is True:
        pad = False
    else:
        pad = "max_length"

    def group_texts(examples):
        # Concatenate all texts.
        concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
        total_length = len(concatenated_examples[list(examples.keys())[0]])
        if total_length >= cfg.dataset.block_size:
            total_length = (
                total_length // cfg.dataset.block_size
            ) * cfg.dataset.block_size
        # Split by chunks of max_len.
        result = {
            k: [
                t[i : i + cfg.dataset.block_size]
                for i in range(0, total_length, cfg.dataset.block_size)
            ]
            for k, t in concatenated_examples.items()
        }
        result["labels"] = result["input_ids"].copy()
        return result

    def tokenize_fn(examples):
        result = tokenizer(
            examples[text_column_name],
            padding=pad,
            truncation=True,
            max_length=cfg.dataset.block_size,
        )
        result["labels"] = result["input_ids"].copy()
        return result

    with accelerator.main_process_first():

        tokenized_datasets = raw_datasets.map(
            tokenize_fn,
            batched=True,
            num_proc=cfg.tokenizer.preprocessing_num_workers,
            load_from_cache_file=not cfg.dataset.overwrite_cache,
            desc="Running tokenizer on dataset",
        )

        if cfg.dataset.concatenate_raw is True:
            tokenized_datasets = tokenized_datasets.map(
                group_texts,
                batched=True,
                num_proc=cfg.tokenizer.preprocessing_num_workers,
                load_from_cache_file=not cfg.dataset.overwrite_cache,
                desc=f"Grouping texts in chunks of {cfg.dataset.block_size}",
            )

    return tokenized_datasets

In [14]:
cfg = OmegaConf.load('conf/config.yaml')

In [19]:
accelerator = create_accelerator(cfg)

/home/paperspace/repos/clm-finetuning/venv/lib/python3.9/site-packages/accelerate/accelerator.py:359: UserWarning: `log_with=wandb` was passed but no supported trackers are currently installed.
  warnings.warn(f"`log_with={log_with}` was passed but no supported trackers are currently installed.")


In [26]:
tokenizer.pad_token = '<PAD>'

In [27]:
tokenized_ds = preprocess(cfg, accelerator, tokenizer, datafile_copy)
#2k examples per second
# 1:30:45 to run

In [29]:
datafile_copy

DatasetDict({
    train: Dataset({
        features: ['text', 'meta'],
        num_rows: 7021438
    })
})

In [28]:
tokenized_ds

DatasetDict({
    train: Dataset({
        features: ['text', 'meta', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 7021438
    })
})

In [30]:
type(tokenized_ds['train'])

datasets.arrow_dataset.Dataset

In [31]:
tokenized_ds['train'].remove_columns(['text', 'meta'])

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 7021438
})

In [32]:
tokenized_ds['train']

Dataset({
    features: ['text', 'meta', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 7021438
})

In [33]:
tokenized_ds_copy = tokenized_ds

In [34]:
tokenized_ds_copy['train'] = tokenized_ds_copy['train'].remove_columns(['text', 'meta'])

In [35]:
tokenized_ds_copy

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 7021438
    })
})

In [36]:
import torch

tokenized_ds_copy.save_to_disk("/mnt/share/the_pile/tokenized")
# torch.save(tokenized_ds_copy, "train_tokenized_00")
# 13:06 to run

In [5]:
from datasets import load_from_disk

tokenized_ds_copy2 = load_from_disk("/mnt/share/the_pile/tokenized")

# How long does this take to run?
# This run time was: 6 min 9 seconds. 2nd run was 5m 48sec
# This run time is important
# Loading in the .zst file was 9 min 45 seconds

In [8]:
tokenized_ds_copy2['train'].train_test_split(0.05)

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 6670366
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 351072
    })
})

In [10]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 7021438
    })
})

In [9]:
tokenized_datasets = tokenized_ds_copy2

In [15]:
if cfg.model.name=='decapoda-research/llama-7b-hf':
    tokenized_datasets = tokenized_datasets['train'].train_test_split(
        test_size=cfg.training.val_split_percent / 100)
else:
    if "train" not in tokenized_datasets.column_names:
        tokenized_datasets = tokenized_datasets['train'].train_test_split(
            test_size=cfg.training.val_split_percent / 100)

train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["test"]

In [16]:
train_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 5968222
})

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer2 = AutoTokenizer.from_pretrained("gpt2")

# model = AutoModelForCausalLM.from_pretrained("gpt2")

/home/paperspace/repos/clm-finetuning/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
tokenizer2.eos_token

'<|endoftext|>'

In [4]:
1==1

True